# Inflation Report

In [ ]:
import datetime
from dateutil.relativedelta import relativedelta
from pyfredapi import FredSeries
from rich import print as rprint
import pandas as pd
import plotly.express as px

In [ ]:
# papermill parameters cell
as_of = datetime.date.today()

In [ ]:
print(f"Report Date: {as_of}")

### Collect data

In [ ]:
client = FredSeries()

In [ ]:
series = [
    "CPIAUCSL",
    "CPILFESL",
    "CUSR0000SETA02",
    "CUSR0000SETA01",
    "CUSR0000SEHA",
    "CUSR0000SEHC",
    "CPIMEDSL",
    "CUSR0000SAF11",
    "CUSR0000SEFV",
    "CUSR0000SAE1",
    "CUSR0000SEEA",
    "CUSR0000SEEB",
    "CPIAPPSL",
]

In [ ]:
responses = {}
dfs = {}

for s in series:
    print(f"Requesting series {s}...")
    response = client.get_series(series_id=s)
    responses[s] = response
    df = responses[s].data
    column_name = (
        response.info.title.lower()
        .replace("consumer price index for all urban consumers: ", "")
        .replace(" in u.s. city average", "")
        .replace("'", "")
        .replace(" ", "_")
    )
    df = df.rename(columns={"value": column_name})
    df = df.drop(["realtime_start", "realtime_end"], axis=1)
    dfs[s] = df.set_index("date")

combined_df = pd.concat(dfs.values(), axis=1)
combined_df = combined_df.loc[:as_of]

## Calculate percent change

In [ ]:
lags = [1, 3, 6, 12, 18, 24, 36, 48, 60]
pct_chg_dfs = []

# calculate the pct change for each lag
for lag in lags:
    pct_chg_df = round(combined_df.pct_change(lag).iloc[[-1]] * 100, 4)

    # set the index to the lag
    pct_chg_df.index = [f"{lag} Month"]

    # append the df to pct_chg_dfs
    pct_chg_dfs.append(pct_chg_df)

# pd.concat dfs together
output = pd.concat(pct_chg_dfs)

In [ ]:
rprint(f"Table for percent changes in CPI series as-of {as_of}")
display(output)

## Visualize change in CPI series over past 5 years

In [ ]:
viz_df = pd.melt(
    combined_df.reset_index(), id_vars="date", value_vars=list(combined_df.columns)
)
viz_df = viz_df.rename({"value": "CPI", "variable": "cpi_series"}, axis=1)

min_date = viz_df[["date"]].min().date.date()
max_date = viz_df[["date"]].max().date.date()

px.line(
    data_frame=viz_df,
    x="date",
    y="CPI",
    color="cpi_series",
    title=f"Change in CPI: {min_date} - {max_date}",
)